In [ ]:
AZURE_RES_NAME=cm-cloud-ws-dev-resources
AZURE_AKS_NAME=cm-cloud-ws-dev-aks

In [ ]:
az aks get-credentials --resource-group $AZURE_RES_NAME --name $AZURE_AKS_NAME
kubectl cluster-info

In [ ]:
# see: https://github.com/grafana/loki
# see: https://hub.helm.sh/charts/stable/grafana
# see: https://github.com/grafana/loki/tree/master/production/helm/loki-stack

kubectl create ns grafana || true
helm repo add loki https://grafana.github.io/loki/charts
helm repo update
helm upgrade --install loki loki/loki-stack \
    --set grafana.ingress.enabled=true,grafana.ingress.annotations."kubernetes\.io/ingress\.class"="addon-http-application-routing",grafana.ingress.path="/",grafana.enabled=true,grafana.plugins={"grafana-piechart-panel"},prometheus.enabled=true,prometheus.ruleNamespaceSelector.any=true,prometheus.prometheusSpec.ruleSelector.matchLabels.role="my-rules",prometheus.alertmanager.persistentVolume.enabled=false,prometheus.server.persistentVolume.enabled=false \
    --namespace="grafana" --wait

# ,grafana.ingress.hosts[0]=$GRAFANA_URL

In [ ]:
GRAFANA_PWD=$(kubectl get secret --namespace grafana loki-grafana -o jsonpath="{.data.admin-password}" | base64 --decode ; echo)
echo "Grafana is installed and you can connect via user 'admin' and password '$GRAFANA_PWD'"
echo ""
echo "if no external traffic routing is installed do following"
echo "run 'kubectl port-forward --namespace grafana service/loki-grafana 3000:80'"
echo "open 'http://localhost:3000/'"

In [ ]:
helm uninstall loki --namespace="grafana"

In [ ]:
kubectl logs -h